In [0]:
df_automobile = spark.read.format("delta").table("silver.automobile")
df_motor      = spark.read.format("delta").table("silver.motor")
df_dimensoes  = spark.read.format("delta").table("silver.dimensoes")
df_origin     = spark.read.format("delta").table("silver.origem")
df_ano_modelo = spark.read.format("delta").table("silver.ano_modelo")

df_automobile.createOrReplaceTempView("automobile")
df_motor.createOrReplaceTempView("motor")
df_dimensoes.createOrReplaceTempView("dimensoes")
df_origin.createOrReplaceTempView("origem")
df_ano_modelo.createOrReplaceTempView("ano_modelo")

In [0]:
%sql
-- Dropar tabela se existir
DROP TABLE IF EXISTS gold.dim_automobile;

-- Criar tabela Gold
CREATE TABLE gold.dim_automobile (
    SK_AUTOMOBILE   BIGINT GENERATED BY DEFAULT AS IDENTITY,
    NAME            VARCHAR(100),
    MPG             DOUBLE,
    CYLINDERS       INT,
    DISPLACEMENT    DOUBLE,
    HORSEPOWER      DOUBLE,
    WEIGHT          DOUBLE,
    ACCELERATION    DOUBLE,
    MODEL_YEAR      INT,
    ORIGIN          VARCHAR(50)
)
USING DELTA;


In [0]:
%sql
-- Merge dados Silver → Gold
WITH automobile_rel AS (
    SELECT a.name,
           a.mpg,
           m.cylinders,
           m.displacement,
           m.horsepower,
           d.weight,
           d.acceleration,
           y.model_year,
           o.origin
      FROM automobile a
           INNER JOIN motor m
             ON a.name = m.name
           INNER JOIN dimensoes d
             ON a.name = d.name
           INNER JOIN ano_modelo y
             ON a.name = y.name
           INNER JOIN origem o
             ON a.name = o.name
)
MERGE INTO gold.dim_automobile AS g
USING automobile_rel AS r
ON g.name = r.name
WHEN MATCHED THEN
    UPDATE SET mpg = r.mpg,
               cylinders = r.cylinders,
               displacement = r.displacement,
               horsepower = r.horsepower,
               weight = r.weight,
               acceleration = r.acceleration,
               model_year = r.model_year,
               origin = r.origin
WHEN NOT MATCHED THEN
    INSERT (name, mpg, cylinders, displacement, horsepower, weight, acceleration, model_year, origin)
    VALUES (r.name, r.mpg, r.cylinders, r.displacement, r.horsepower, r.weight, r.acceleration, r.model_year, r.origin);


In [0]:
%sql
DROP TABLE IF EXISTS gold.dim_motor;

CREATE TABLE gold.dim_motor (
    SK_MOTOR       BIGINT GENERATED BY DEFAULT AS IDENTITY,
    NAME           VARCHAR(100),
    CYLINDERS      INT,
    DISPLACEMENT   DOUBLE,
    HORSEPOWER     DOUBLE
)
USING DELTA;


In [0]:
%sql
WITH motor_rel AS (
    SELECT name, cylinders, displacement, horsepower
    FROM motor
)
MERGE INTO gold.dim_motor AS g
USING motor_rel AS r
ON g.name = r.name
WHEN MATCHED THEN
    UPDATE SET cylinders = r.cylinders,
               displacement = r.displacement,
               horsepower = r.horsepower
WHEN NOT MATCHED THEN
    INSERT (name, cylinders, displacement, horsepower)
    VALUES (r.name, r.cylinders, r.displacement, r.horsepower);


In [0]:
%sql
DROP TABLE IF EXISTS gold.dim_dimensoes;

CREATE TABLE gold.dim_dimensoes (
    SK_DIM      BIGINT GENERATED BY DEFAULT AS IDENTITY,
    NAME        VARCHAR(100),
    WEIGHT      DOUBLE,
    ACCELERATION DOUBLE
)
USING DELTA;


In [0]:
%sql
WITH dim_rel AS (
    SELECT name, weight, acceleration
    FROM dimensoes
)
MERGE INTO gold.dim_dimensoes AS g
USING dim_rel AS r
ON g.name = r.name
WHEN MATCHED THEN
    UPDATE SET weight = r.weight,
               acceleration = r.acceleration
WHEN NOT MATCHED THEN
    INSERT (name, weight, acceleration)
    VALUES (r.name, r.weight, r.acceleration);


In [0]:
%sql
DROP TABLE IF EXISTS gold.dim_origem;

CREATE TABLE gold.dim_origem (
    SK_ORIGEM BIGINT GENERATED BY DEFAULT AS IDENTITY,
    NAME      VARCHAR(100),
    ORIGIN    VARCHAR(50)
)
USING DELTA;


In [0]:
%sql
WITH origem_rel AS (
    SELECT name, origin
    FROM origem
)
MERGE INTO gold.dim_origem AS g
USING origem_rel AS r
ON g.name = r.name
WHEN MATCHED THEN
    UPDATE SET origin = r.origin
WHEN NOT MATCHED THEN
    INSERT (name, origin)
    VALUES (r.name, r.origin);


In [0]:
%sql
DROP TABLE IF EXISTS gold.dim_ano_modelo;

CREATE TABLE gold.dim_ano_modelo (
    SK_ANO      BIGINT GENERATED BY DEFAULT AS IDENTITY,
    NAME        VARCHAR(100),
    MODEL_YEAR  INT
)
USING DELTA;


In [0]:
%sql
WITH ano_rel AS (
    SELECT name, model_year
    FROM ano_modelo
)
MERGE INTO gold.dim_ano_modelo AS g
USING ano_rel AS r
ON g.name = r.name
WHEN MATCHED THEN
    UPDATE SET model_year = r.model_year
WHEN NOT MATCHED THEN
    INSERT (name, model_year)
    VALUES (r.name, r.model_year);
